
# Experiment motivation

The idea is to adjust hyperparameters and understand if performance improves. Number of layers increase to 5 from 3. Embedding size to 256 from 64. Lr decreases to 0.001 from 0.01 among other changes.



In [ ]:
args = {
    'num_layers': 5,
    'hidden_dim': 256,
    'output_dim': 256,
    'dropout': 0.5,
    'lr': 0.001,
    'epochs': 25,
    'num_vocab': 5000,
    'warmup': 2,
    'weight_decay': 0.000001,
    'words_per_method_name': 5,
    'max_steps': 99999,
    'batch_size': 32,
}

# Setup

In [ ]:
!pip install ogb
!pip install torch_geometric
!python -c "import ogb; print(ogb.__version__)"

import os
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader
import torch
import pandas as pd
import torch.nn.functional as F
from tqdm.notebook import tqdm
print(torch.__version__)

# The PyG built-in GCNConv
from torch_geometric.nn import GCNConv

import torch_geometric.transforms as T
from torch_geometric.nn import global_add_pool, global_mean_pool
from numpy import float32
# from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

# codeBERT = RobertaModel.from_pretrained("microsoft/codebert-base")
# codeBERT.to(device)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
import copy
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=b3e14ddd6ada108a50313c7226d6fba3f5a6fc05b1762ec312eb218f61c808b0
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.4 MB/s eta 0:00:00
1.3.6
2.1.0+cu118
Device: cuda


# Dataset

In [ ]:
dataset = PygGraphPropPredDataset(name = "ogbg-code2")
print(dataset)
evaluator = Evaluator(name='ogbg-code2')
'''
==== Expected input format of Evaluator for ogbg-code2
{'seq_ref': seq_ref, 'seq_pred': seq_pred}
- seq_ref: a list of lists of strings
- seq_pred: a list of lists of strings
where seq_ref stores the reference sequences of sub-tokens, and
seq_pred stores the predicted sequences of sub-tokens.

==== Expected output format of Evaluator for ogbg-code2
{'F1': F1}
- F1 (float): F1 score averaged over samples.
'''
# print(dataset.num_classes)
# print(dataset.num_node_features)
# print(dataset[0])
# print(len(dataset.y))

Downloaded 0.91 GB: 100%|██████████| 934/934 [00:30<00:00, 30.92it/s]


Extracting dataset/code2.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 452741/452741 [00:01<00:00, 338942.42it/s]


...

100%|██████████| 452741/452741 [00:19<00:00, 22908.03it/s]


Saving...


Done!


PygGraphPropPredDataset(452741)


"\n==== Expected input format of Evaluator for ogbg-code2\n{'seq_ref': seq_ref, 'seq_pred': seq_pred}\n- seq_ref: a list of lists of strings\n- seq_pred: a list of lists of strings\nwhere seq_ref stores the reference sequences of sub-tokens, and\nseq_pred stores the predicted sequences of sub-tokens.\n\n==== Expected output format of Evaluator for ogbg-code2\n{'F1': F1}\n- F1 (float): F1 score averaged over samples.\n"

# GCN Model
This GCN model uses multiple GCN layers and a global_add_pool pooling method for graph level aggregation.

In [ ]:
from torch_geometric.nn.models.tgn import Linear
class GCN(torch.nn.Module):
    """
    AST is a directed graph with 100-150 nodes. This GCN takes in nodes of 2 dimension
    features and convert them to 1x256 embeddings using a basic encoder ASTNodeEncoder
    before passing the embedding through GCN layers. No self-loops are added to establish
    a baseline. Additional augmentation and words are added by using codeBERT to transform
    them into dense embeddings

    Input
        Batch of node_size x256 dimensions
    Output
        Embeddings of batch_size x pred_list_len X num_classes, i.e. 64 x 5 x len(vocab_list)
    """
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, pred_list_len, num_classes):
        super(GCN, self).__init__()

        self.hidden_dim = hidden_dim
        self.pred_list_len = pred_list_len
        self.num_classes = num_classes

        # GCN layers
        self.convs = torch.nn.ModuleList()
        self.convs.extend([GCNConv(hidden_dim, hidden_dim) for i in range(num_layers-1)])
        self.convs.append(GCNConv(hidden_dim, output_dim))

        # Batch norm
        self.bns = torch.nn.ModuleList()
        self.bns.extend([torch.nn.BatchNorm1d(hidden_dim) for i in range(num_layers-1)])

        # Global pool
        self.pool_method = global_add_pool

        # Probability of an element getting zeroed
        self.dropout = dropout

        # Post-processing layer that transform graph embedding into 5 token
        # predictions for each graph
        self.post_mps = Linear(output_dim, num_classes*pred_list_len, bias=True)

        # self.post_mps = torch.nn.ModuleList()
        # for i in range(pred_list_len):
        #     self.post_mps.append(torch.nn.Linear(output_dim, self.num_classes))

        self.softmax = torch.nn.Softmax(dim=-1)

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, batched_data):
        x, edge_index, batch = batched_data.x, batched_data.edge_index, batched_data.batch

        # Pre-processing. Each node x has two features initially, the node type (e.g for , while)
        # and the node text (e.g. delete). ASTNodeEncoder is used to turn
        # 2 dimension into dim_hidden dimensions through a linear transformation

        # print("Raw x shape {}".format(x.shape)) # batch size * N x D

        embedding = torch.nn.Sequential(
            Linear(x.shape[1], self.hidden_dim)  # 2 input features, 512 output features
        )
        embedding.to(device)
        x = embedding(x.to(dtype=torch.float32)) # batch_size * N x D'
        # print("x embedding {}".format(x.shape))

        out = None
        for i in range(len(self.convs)-1):
            x = self.convs[i](x, edge_index)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        out = self.convs[-1](x, edge_index) # batch * N x outdim

        out = self.pool_method(out, batch) # batch x outdim
        #print("Output before pooling {}".format(out.shape))

        # We want to turn one single graph embedding per graph to pred 5
        # words. So the desired output is batch_size x word len x num_classes
        out = self.post_mps(out).view(out.shape[0], self.pred_list_len, self.num_classes) # batch_size x word_len x num_classes

        # pred_list = []
        # for i in range(self.pred_list_len):
        #     pred_list.append(self.post_mps[i](out))
        # out = torch.stack(pred_list, dim=1)

        #out = self.softmax(out)
        #print(out)

        return out

# Train

In [ ]:
def train(model, device, data_loader, optimizer, epoch, args):
    model.train()

    loss_accum = 0

    # Used to shorten training for debugging
    max_steps = args['max_steps']

    loss_fn = torch.nn.CrossEntropyLoss()

    for step, batch in enumerate(tqdm(data_loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
            pass
        else:
            # if epoch < args['warmup']:
            #     iteration = epoch * len(data_loader) + step
            #     for param_group in optimizer.param_groups:
            #         param_group["lr"] = lr_modifier(iteration)

            # Each prediction is the one-hot probability of the classification
            pred_list = model(batch) # batch_size x words per graph x num_classes
            optimizer.zero_grad()

            # y comes in as a list of words per graph. Need to one-hot encode it
            y = one_hot_y(batch.y, args['num_classes'], args['words_per_method_name']) #batch_size x words_per_method_name x num_classes i.e. 32x5x15002

            # print("Pred_list {}".format(pred_list.shape))
            # print("One-hot encoded y shape {}".format(y.shape))

            #print("Batch y {}".format(batch.y[0]))

            # print("Pred_list {}".format(pred_list[0]))
            # print("One-hot encoded y {}".format(y[0]))

            loss = 0
            for i in range(pred_list.shape[0]):
                loss += loss_fn(pred_list[i].to(torch.float32), y[i].to(torch.float32))


            # pred_flat = pred_list.view(pred_list.shape[0]*args['words_per_method_name'],
            #                            args['num_classes']) # batch*word len x num_class
            # y_flat = y.view(y.shape[0]*args['words_per_method_name'],
            #                            args['num_classes']) # batch*word len x num_class

            #loss = loss_fn(pred_flat, y_flat.to(torch.float32))

            # print("Pred flat {}".format(pred_flat))
            # print("Y_flat {}".format(y_flat))

            loss_accum += loss/pred_list.shape[0]

            #print("Loss {}".format(loss.item()))

            loss.backward()
            optimizer.step()

            # Used to shorten training for debugging
            if step > max_steps:
                break

    print("Epoch loss {}".format(loss_accum / (step+1)))


# Eval

In [ ]:
def eval(model, device, loader, evaluator, args):
    model.eval()
    seq_ref_list = []
    seq_pred_list = []

    # Used to shorten training for debugging
    max_steps = args['max_steps']
    step_count = 0

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred_list = model(batch) # batch_size x pred_list_len X num_classes

            #print("Preds are {}".format(pred_list.shape))
            seq_ref_list.extend(batch.y)
            seq_pred_list.extend(decode_to_word(pred_list))

            #print("Seq_ref_list len {} pred list len {}".format(len(seq_ref_list), len(seq_pred_list)))

        # Used to shorten for debugging
        step_count += 1
        if step_count > max_steps:
            break

    print("seq_ref_list {}".format(seq_ref_list[:5]))
    print("seq_pred_list {}".format(seq_pred_list[:5]))

    input_dict = {"seq_ref": seq_ref_list, "seq_pred": seq_pred_list}

    return evaluator.eval(input_dict)

# Helper methods
Might modularize away in a different file

In [ ]:
class ASTNodeEncoder(torch.nn.Module):
    '''
        Input:
            x: Default node feature. Each node in the ogbg-code2 graph has 2 features.
               One is the node type ID (e.g. while) and the other the text id (e.g. delete)
            depth: The depth of the node in the AST.
        Output:
            emb_dim-dimensional vector
    '''
    def __init__(self, x, emb_dim):
        super(ASTNodeEncoder, self).__init__()
        self.embedding = torch.nn.Embedding(len(x), emb_dim)

    def forward(self, x):
        return self.embedding(x)


def get_vocab_mapping(seq_list, num_vocab):
    '''
        Input:
            seq_list: a list of sequences
            num_vocab: vocabulary size
        Output:
            vocab2idx:
                A dictionary that maps vocabulary into integer index.
                Additioanlly, we also index '__UNK__' and '__EOS__'
                '__UNK__' : out-of-vocabulary term
                '__EOS__' : end-of-sentence
            idx2vocab:
                A list that maps idx to actual vocabulary.
    '''

    vocab_cnt = {}
    vocab_list = []
    for seq in seq_list:
        for w in seq:
            if w in vocab_cnt:
                vocab_cnt[w] += 1
            else:
                vocab_cnt[w] = 1
                vocab_list.append(w)

    cnt_list = np.array([vocab_cnt[w] for w in vocab_list])
    topvocab = np.argsort(-cnt_list, kind = 'stable')[:num_vocab]

    print('Coverage of top {} vocabulary:'.format(num_vocab))
    print(float(np.sum(cnt_list[topvocab]))/np.sum(cnt_list))

    vocab2idx = {vocab_list[vocab_idx]: idx for idx, vocab_idx in enumerate(topvocab)}
    idx2vocab = [vocab_list[vocab_idx] for vocab_idx in topvocab]

    # print(topvocab)
    # print([vocab_list[v] for v in topvocab[:10]])
    # print([vocab_list[v] for v in topvocab[-10:]])

    vocab2idx['__UNK__'] = num_vocab
    idx2vocab.append('__UNK__')

    vocab2idx['__EOS__'] = num_vocab + 1
    idx2vocab.append('__EOS__')

    # test the correspondence between vocab2idx and idx2vocab
    for idx, vocab in enumerate(idx2vocab):
        assert(idx == vocab2idx[vocab])

    # test that the idx of '__EOS__' is len(idx2vocab) - 1.
    # This fact will be used in decode_arr_to_seq, when finding __EOS__
    assert(vocab2idx['__EOS__'] == len(idx2vocab) - 1)

    return vocab2idx, idx2vocab

def one_hot_y(batch_y, num_classes, max_seq_len):
    '''
    Given data.y, produce a tensor batch_size x number of words per method name.

    Input:
        batch_data.y: a list of lists, each of word len. batch_size x word len
        output: one hot encoding of dimensions batch_size x word len x num_classes.
    '''
    #print(batch_y)
    labels = torch.zeros(len(batch_y), max_seq_len, dtype=torch.long).to(device)

    for i, y in enumerate(batch_y):
        # if the number of words in y is less than the max length, fill blanks with end of word token
        y = y[:max_seq_len] + ['__EOS__'] * max(0, max_seq_len - len(y))
        for j, w in enumerate(y):
            index = vocab2idx['__UNK__']
            if w in vocab2idx:
                index = vocab2idx[w]
            labels[i][j] = index
        # print(labels[i])
    labels_one_hot = F.one_hot(labels, num_classes).to(device) # batch x word len x num_class
    #print('labels {}'.format(labels_one_hot))

    return labels_one_hot

def decode_to_word(pred_list):
    '''
        The goal is to turn an one-hot-encoder back into word. We need to take the
        argmax of the prediction

        Input:  embeddings, batch x word len x num class
        Output: list of lists, each a sequence of words. batch x word len
    '''
    #print("Prediction {}".format(pred_list))
    argmax = torch.argmax(pred_list, dim=-1).tolist()
    #print("Argmax {}".format(argmax))

    for i, graph in enumerate(argmax):
        word_list = []
        for word_index in graph:
            if word_index < len(vocab2idx) - 2:
                w = idx2vocab[word_index]
                word_list.append(w)
        argmax[i] = word_list
    #print("decoded pred words {}".format(argmax))
    return argmax

#Run Epochs
Train and then evaluate the accuracy of the GCN model

In [ ]:
def run_epochs(model, evaluator, args, dataset):
    split_idx = dataset.get_idx_split()
    train_loader = DataLoader(dataset[split_idx["train"]], batch_size=args['batch_size'], shuffle=True)
    valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=args['batch_size'], shuffle=False)
    test_loader = DataLoader(dataset[split_idx["test"]], batch_size=args['batch_size'], shuffle=False)

    valid_curve = []
    test_curve = []
    train_curve = []

    optimizer = torch.optim.AdamW(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

    # # We want to decrease lr as training goes progresses.
    # # lr_scheduler method decays lr with a cosine annealing according to https://arxiv.org/pdf/1608.03983.pdf
    # lr_modifier = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args['epochs'])
    # lr_steps = args['lr'] / (args['warmup'] * len(train_loader))
    # def lr_scheduler(s):
    #     lr = s * lr_steps
    #     return lr

    for epoch in range(1, args['epochs'] + 1):
        print("=====Epoch {}".format(epoch))
        print('Training...')
        train(model, device, train_loader, optimizer, epoch, args)

        print('Evaluating...')
        train_perf = eval(model, device, train_loader, evaluator, args)
        valid_perf = eval(model, device, valid_loader, evaluator, args)
        test_perf = eval(model, device, test_loader, evaluator, args)

        # if epoch >= args['warmup'] and lr_modifier is not None:
        #     lr_modifier.step()

        print({'Train': train_perf, 'Validation': valid_perf, 'Test': test_perf})

        train_curve.append(train_perf[dataset.eval_metric])
        valid_curve.append(valid_perf[dataset.eval_metric])
        test_curve.append(test_perf[dataset.eval_metric])

    print('F1')
    best_val_epoch = np.argmax(np.array(valid_curve))
    best_train = max(train_curve)
    print('Finished training!')
    print('Best validation score: {}'.format(valid_curve[best_val_epoch]))
    print('Test score: {}'.format(test_curve[best_val_epoch]))

In [ ]:
# There are 450k words (duplicates included) in the true labels.
# The top 15k is extracted and stored in a dict vocab2idx.
# This top 15k covers 96% of the all unique words.
vocab2idx, idx2vocab = get_vocab_mapping(dataset.y, args['num_vocab'])
print("Vocab list {}".format(vocab2idx))
args['num_classes'] = len(vocab2idx)

Coverage of top 5000 vocabulary:
0.901200162173439
Vocab list {'get': 0, 'to': 1, 'set': 2, 'create': 3, 'from': 4, 'add': 5, 'parse': 6, 'update': 7, 'list': 8, 'file': 9, 'check': 10, 'data': 11, 'is': 12, 'load': 13, 'read': 14, 'run': 15, 'by': 16, 'delete': 17, 'find': 18, 'write': 19, 'for': 20, 'name': 21, 'make': 22, 'remove': 23, 'dict': 24, 'all': 25, 'build': 26, 'config': 27, 'process': 28, 'path': 29, 'validate': 30, 'send': 31, 'on': 32, 'request': 33, 'save': 34, 'value': 35, 'type': 36, 'id': 37, 'generate': 38, 'info': 39, 'and': 40, 'user': 41, 'handle': 42, 'url': 43, 'init': 44, 'key': 45, 'string': 46, 'start': 47, 'with': 48, 'filter': 49, 'json': 50, 'convert': 51, 'query': 52, 'format': 53, 'main': 54, 'register': 55, 'plot': 56, 'in': 57, 'output': 58, 'event': 59, 'search': 60, 'model': 61, 'state': 62, 'message': 63, 'log': 64, 'index': 65, 'group': 66, 'table': 67, 'do': 68, 'extract': 69, 'as': 70, 'default': 71, 'text': 72, 'image': 73, 'object': 74, 'map'

# Main
Grab ogbg-code2 and create a dictionary of all of the y labels

In [ ]:
if __name__ == "__main__":
    gcn = GCN(
        dataset[0].num_features,
        args['hidden_dim'],
        args['output_dim'],
        args['num_layers'],
        args['dropout'],
        args['words_per_method_name'],
        args['num_classes'],
    ).to(device)

    # param_size = 0
    # for param in gcn.parameters():
    #     param_size += param.nelement() * param.element_size()
    # print(param_size)

    run_epochs(gcn, evaluator, args, dataset)

=====Epoch 1
Training...


Iteration:   0%|          | 0/12750 [00:00<?, ?it/s]

Epoch loss 4.108046531677246
Evaluating...


Iteration:   0%|          | 0/12750 [00:01<?, ?it/s]

seq_ref_list [['anscombe'], ['const', 'compare', 'digest', 'backport'], ['number', 'of', 'y', 'tiles', 'at', 'zoom'], ['is', 'partial'], ['vi', 'pos', 'matching']]
seq_pred_list [['vcenter', 'nameserver', 'detail', 'protocol', 'rules'], ['vcenter', 'nameserver', 'detail', 'protocol', 'rules'], ['vcenter', 'nameserver', 'detail', 'protocol', 'rules'], ['vcenter', 'nameserver', 'detail', 'protocol', 'rules'], ['vcenter', 'nameserver', 'detail', 'protocol', 'rules']]


...

seq_ref_list [['detect', 'file', 'triggers'], ['p', 'function', 'nowidth'], ['edit', 'profile'], ['vfr', 'hud', 'encode'], ['get', 'action', 'type']]
seq_pred_list [['objfn', 'provide', 'restrict', 'foreground', 'fetchmany'], ['objfn', 'provide', 'restrict', 'foreground', 'fetchmany'], ['objfn', 'provide', 'restrict', 'foreground', 'fetchmany'], ['assignments', 'provide', 'versioned', 'foreground', 'fetchmany'], ['objfn', 'provide', 'restrict', 'foreground', 'fetchmany']]


Iteration:   0%|          | 0/714 [00:00<?, ?it/s]

seq_ref_list [['learn'], ['save', 'act'], ['nature', 'cnn'], ['mlp'], ['lstm']]
seq_pred_list [['objfn', 'provide', 'restrict', 'foreground', 'fetchmany'], ['objfn', 'provide', 'restrict', 'foreground', 'fetchmany'], ['objfn', 'provide', 'restrict', 'foreground', 'fetchmany'], ['objfn', 'provide', 'restrict', 'foreground', 'fetchmany'], ['objfn', 'provide', 'restrict', 'foreground', 'fetchmany']]


Iteration:   0%|          | 0/686 [00:00<?, ?it/s]

seq_ref_list [['get', 'vid', 'from', 'url'], ['sina', 'xml', 'to', 'url', 'list'], ['make', 'mimi'], ['fc2video', 'download'], ['dailymotion', 'download']]
seq_pred_list [['objfn', 'provide', 'versioned', 'foreground', 'fetchmany'], ['objfn', 'provide', 'restrict', 'foreground', 'fetchmany'], ['assignments', 'provide', 'versioned', 'foreground', 'fetchmany'], ['objfn', 'provide', 'restrict', 'foreground', 'fetchmany'], ['objfn', 'provide', 'restrict', 'foreground', 'fetchmany']]
{'Train': {'precision': 6.323901405965056e-05, 'recall': 0.00017055741841023327, 'F1': 8.750744639620455e-05}, 'Validation': {'precision': 8.765394223605207e-06, 'recall': 1.4608990372675343e-05, 'F1': 1.0956742779506508e-05}, 'Test': {'precision': 8.201202843083653e-05, 'recall': 0.00023540489642184554, 'F1': 0.00011405352438703998}}
F1
Finished training!
Best validation score: 0.0012909888901474122
Test score: 0.0013672320328963305
